<a href="https://colab.research.google.com/github/james-bowden/teaching/blob/master/scraping_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Web Data via Scraping and Automation

### Part 2: But how do I extract the data I want?

James Bowden, 07/2020. This tutorial is the second of a two part series on how to scrape web data. The first can be found here (https://colab.research.google.com/drive/1KH18f5y9rbvEga6aDdUV907E-rxw453D?usp=sharing). This series was written as a part of Caltech's CS42 course (Computer Science Education).

#### What next?

In the first part, we learned how to find web data and download it! Now that we've gotten our data, we need to figure out how to extract just the information we want. We do this by **parsing** the data. Note that we will want to be able to parse HTML data as well as other kinds of data. Data comes in all shapes and sizes and is often *not* in the format you'd like it to be in. In order to be able to use all sorts of data, we need to understand how to parse anything. Once we know how to get the data we want, we can create functions to do it automatically and package it into a nice command tool for ease of use.

#### What is parsing, and how do I do it?

**Parsing** is a technique by which we separate out data we want based on patterns. The basic idea is that if we can identify the data we want with a **unique pattern or identifier**, then we can separate it out from everything else by searching for this particular pattern and only taking things that come after it (or before it, or are it, etc.). 

Parsing is usually done on strings of characters, which HTML code happens to be. Most data is stored as a sequence of characters, and then separated out via parsing. One common example is text files, which consist of characters with newlines (denoted by '\n') throughout them that signal to the program when to start a newline. Another example is .csv files, which consist of values separated by commas (and newline characters) so that a program like Excel can find the commas and put each separate value into its own cell. Let's try writing our own basic function to parse a string.

As an example, let's take a string consisting of a few sentences. I'd like to identify all of the things that the writer did. Now obviously, the English language is very complicated and there are many ways to express actions, but for this example we'll just count verbs that come immediately after 'I', such as 'today I biked to school', in which case 'biked' would be the desired verb. Note that when used in this context, 'I' is always followed by a space, so we should parse for 'I '. 

The first thing we need to do is just find all of the instances of 'I ', and denote their location. We'll write a simple function that does just that: takes in a string to search and a string to search for, and returns a list of their indices. 

In [ ]:
def find(subject, search):
    '''Given a subject string to search and another string to search for, returns a list of their starting indices.'''
    indices = []
    # we'll need the length of the search string to know how many characters to compare against and to skip once we find it
    length = len(search)
    ind = 0
    # want to search from the start of the subject until the last possible match, which is found 
    # length characters before the end (has to be space for the search string to fit!)
    for x in range(0, len(subject) - length):
        # 'slice' the string to get only the first length characters.
        s_compare = subject[ind:(ind + length)]
        # if s_compare equals search, record the index and skip to the end of the search string!
        if s_compare == search:
            indices.append(ind)
            ind += length
        # if not, move forward one character in the subject string
        else:
            ind += 1
    return indices

In [ ]:
# test case
paragraph = 'My name is James. Today I ate pizza for lunch. After that, I worked on some homework. I called my friends to see how they were doing. Then I napped for a few hours.'
search = 'I '
# call our function:
indices = find(paragraph, search)
print(indices)
# make sure there is an 'I' at these indices
for index in indices:
    print(paragraph[index:index + len(search)])

[24, 59, 86, 139]
I 
I 
I 
I 


We see that our function is indeed finding all of the occurrences of 'I '. Great! Now let's write a function that can take the next word after 'I'. We'll define a word as ending in a space or a period, though in reality there are a bunch of other punctuations symbols to check for. It's a bit more convenient to just call the find() method we wrote from within extract() since we always need to find the indices.

In [ ]:
def extract(subject, search):
    '''Returns list of words in subject following the search string.'''
    # get indices
    indices = find(subject, search)
    words = []
    for index in indices:
        # skip over the search string since we don't actually want that
        start = index + len(search)
        # set end index at end of subject string by default
        end = len(subject)
        # from start to end, check each character
        for x in range(start, end):
            # if we find a space or period, we know that our word ends here! exit the for loop.
            if subject[x] == ' ' or subject[x] == '.':
                end = x
                break
        # get next word by slicing from start to end
        next_word = subject[start:end]
        words.append(next_word)
    return words

In [ ]:
# test on same case again:
verbs = extract(paragraph, search)
print(verbs)

['ate', 'worked', 'called', 'napped']


And now we have the verbs that follow 'I'. I could use this to collect all of the things somebody has done from their diary or lab notebook automatically, which is...very useful!? Though this example is basic, it's good to understand what exactly is going on when we parse strings. If we wanted to parse based on more rules (for example, accounting for newlines, or taking the verbs after pronouns), it wouldn't be difficult to add in a few more conditions or parse multiple times.

There are functions built into Python that you can use to do these things, but they can't cover all cases. We'll see this in just a moment when we try to parse some of our data files. A few of these standard functions are shown below to illustrate their use, but you can Google them as needed. 

In [ ]:
paragraph = '''My name is James. Today I ate pizza for lunch. After that, I worked on some homework. I called my friends to see how they were doing. Then I napped for a few hours.'''
search = 'I '
# builtin find method gets 1st occurrence
first_index = paragraph.find(search)
print(first_index)
print(paragraph[first_index:first_index + len(search)])
# builtin split method--very useful for extracting certain parts of a string
# here we split the paragraph by spaces:
words = paragraph.split(' ')
print('Split by \' \':')
print(words)
# or by periods:
words = paragraph.split('.')
print('Split by \'.\':')
print(words)
# or by 'I 's, which we could use by just taking the first word of each list entry except the first.
words = paragraph.split('I ')
print('Split by \'I \':')
print(words)

24
I 
Split by ' ':
['My', 'name', 'is', 'James.', 'Today', 'I', 'ate', 'pizza', 'for', 'lunch.', 'After', 'that,', 'I', 'worked', 'on', 'some', 'homework.', 'I', 'called', 'my', 'friends', 'to', 'see', 'how', 'they', 'were', 'doing.', 'Then', 'I', 'napped', 'for', 'a', 'few', 'hours.']
Split by '.':
['My name is James', ' Today I ate pizza for lunch', ' After that, I worked on some homework', ' I called my friends to see how they were doing', ' Then I napped for a few hours', '']
Split by 'I ':
['My name is James. Today ', 'ate pizza for lunch. After that, ', 'worked on some homework. ', 'called my friends to see how they were doing. Then ', 'napped for a few hours.']


#### Parsing HTML with BeautifulSoup

Back to our example from last time: we have a URL and we've downloaded the HTML code from the server. It's now stored in our variable, and we need to parse it. 

In [ ]:
import requests

# get HTML from url
url = 'https://www.ncbi.nlm.nih.gov/assembly/GCF_000005845.2'
response = requests.get(url)
print('Preview:')
print(response.text[:500])

Preview:
<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">
    <head xmlns:xi="http://www.w3.org/2001/XInclude"><meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <!-- meta -->
    <meta name="robots" content="index,nofollow,noarchive" />
<meta name="ncbi_app" content="entrez" /><meta name="ncbi_db" conten


In the first tutorial, we inspected the NCBI webpage of an *E. coli* strain (https://www.ncbi.nlm.nih.gov/assembly/GCF_000008865.2) to figure out where the data we wanted was stored in the HTML code. We want to extract the 'Total sequence length', and then extract the url for the 'FTP directory for RefSeq assembly', which contains several links to downloads. We can request this url the same way we did the first, parse its HTML code, and then download the data for coding genes to use locally.

The first step is parsing the HTML code. From inspecting, we know that the total sequence length is stored here, in a table:

![](https://drive.google.com/uc?export=view&id=1Nx-uR7wSxAxFFqP7aUGYFy8ectC-Ts8B)

While we could parse this ourselves, Python has a package that will do a lot of it for us! The package is called BeautifulSoup and you should install it now if you don't already have it.

In [ ]:
# installation command for miniconda/anaconda environment
conda install -c anaconda beautifulsoup4

BeautifulSoup simplifies the parsing of HTML code by dealing with a lot of the formatting internally and picking out the tags and attributes that we want. We first import the package and initialize BeautifulSoup with an HTML parser on the content of the HTML code we've downloaded from the web server.

In [ ]:
from bs4 import BeautifulSoup

# parse HTML and store in variable
soup = BeautifulSoup(response.content, 'html.parser')

We now have a variable named soup that contains all of the HTML data in a format that can be easily parsed by BeautifulSoup. We can use the find() method to search for tables. Our unique identifier here is going to be the table's summary attribute, which equals 'Global statistics'.

In [ ]:
# parse HTML for 'td' tag
table = soup.find('table', {'summary':'Global statistics'})
print(table)

<table class="margin_t0 jig-ncbigrid" summary="Global statistics"><tbody><tr><td>Total sequence length</td><td class="align_r">4,641,652</td></tr><tr><td>Total ungapped length</td><td class="align_r">4,641,652</td></tr><tr><td>Total number of chromosomes and plasmids</td><td class="align_r">1</td></tr></tbody></table>


And now that we have the table, we can pull out the total sequence length pretty easily by getting all of the table entries (denoted by 'td') and then taking the one that is after 'Total sequence length'.

In [ ]:
# get all entries in the table
tds = table.find_all('td')
print(tds)

[<td>Total sequence length</td>, <td class="align_r">4,641,652</td>, <td>Total ungapped length</td>, <td class="align_r">4,641,652</td>, <td>Total number of chromosomes and plasmids</td>, <td class="align_r">1</td>]


In [ ]:
# extract the length after total sequence length string has been found
ind = 0
for entry in tds:
    if 'Total sequence length' == entry.text:
        # get the next entry!
        length = tds[ind + 1].text
        break
print(length)

4,641,652


Just like that, we've got the length! Now we can apply a similar process to get the hyperlink for the FTP directory by parsing for a distinct part of the displayed link name ('FTP directory for RefSeq assembly') in all of the hyperlinks and then extracting the url once we've found it with link['href'].

In [ ]:
links = soup.find_all('a')
for link in links:
    if 'RefSeq assembly' in link.text:
        # extract url
        ftp_url = link['href']
print(ftp_url)

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2


#### Getting FTP via urllib

We'll do this process one more time to get the link to the coding genes file, now requesting the FTP url. This is a good real world example of the fact that data doesn't always come in the format that we want it in. As a result, we'll have to try some other packages and parsing in order to extract what we want.

The requests package we've been using is good for almost every site you'll want to go to, but unfortunately doesn't support FTP. We're going to use the urllib package in this case, which is a bit more complicated but does essentially the same thing as requests. Situations like this come up often when web scraping--just search Google or Stack Overflow for the problem you're having, try different solutions, and make sure to read the documentation for packages you're trying to use if you keep getting errors.

In [ ]:
# installation command for miniconda/anaconda environment
conda install -c ulmo urllib3

In [ ]:
import urllib.request

# make a request object
ftp = urllib.request.Request(ftp_url)
# open, read, and decode the object
with urllib.request.urlopen(ftp) as f:
    ftp_html = f.read().decode()
print(ftp_html)

-r--r--r--   1 ftp      anonymous     1208 Feb 11 20:00 GCF_000005845.2_ASM584v2_assembly_report.txt
-r--r--r--   1 ftp      anonymous     3746 Feb 11 20:00 GCF_000005845.2_ASM584v2_assembly_stats.txt
-r--r--r--   1 ftp      anonymous  1504909 Oct 14  2018 GCF_000005845.2_ASM584v2_cds_from_genomic.fna.gz
-r--r--r--   1 ftp      anonymous      244 Oct 14  2018 GCF_000005845.2_ASM584v2_feature_count.txt.gz
-r--r--r--   1 ftp      anonymous   230172 Oct 14  2018 GCF_000005845.2_ASM584v2_feature_table.txt.gz
-r--r--r--   1 ftp      anonymous  1379902 Oct 31  2014 GCF_000005845.2_ASM584v2_genomic.fna.gz
-r--r--r--   1 ftp      anonymous  3462090 Mar  2  2019 GCF_000005845.2_ASM584v2_genomic.gbff.gz
-r--r--r--   1 ftp      anonymous   465467 Feb 11 20:00 GCF_000005845.2_ASM584v2_genomic.gff.gz
-r--r--r--   1 ftp      anonymous   483693 Feb 11 20:00 GCF_000005845.2_ASM584v2_genomic.gtf.gz
-r--r--r--   1 ftp      anonymous   898290 Oct 14  2018 GCF_000005845.2_ASM584v2_protein.faa.gz
-r--r--r-

BeautifulSoup is great for parsing HTML data! But as you can see, this isn't in HTML format--instead, it looks like directory information was just printed out. Because this isn't standard HTML data, we'll have to parse it a bit differently, which is why it's important to understand how to parse strings in general.

It appears that this is all one long string, so we can just separate it by spaces. However, we can see that here there are multiple spaces to filter out in between different items. We could do this ourselves with a while loop, but we'll use Python's built in re (regular expressions) package because it's easier. Regular expressions are very useful, but also quite complicated and could be a whole separate tutorial. I'd recommend learning how to use them whenever you have a chance though, as they make parsing *much* simpler!

In [ ]:
import re

# split by spaces; the + denotes that spaces next to each other will also be removed
items = re.split(' +', ftp_html)
print(items)

['-r--r--r--', '1', 'ftp', 'anonymous', '1208', 'Feb', '11', '20:00', 'GCF_000005845.2_ASM584v2_assembly_report.txt\r\n-r--r--r--', '1', 'ftp', 'anonymous', '3746', 'Feb', '11', '20:00', 'GCF_000005845.2_ASM584v2_assembly_stats.txt\r\n-r--r--r--', '1', 'ftp', 'anonymous', '1504909', 'Oct', '14', '2018', 'GCF_000005845.2_ASM584v2_cds_from_genomic.fna.gz\r\n-r--r--r--', '1', 'ftp', 'anonymous', '244', 'Oct', '14', '2018', 'GCF_000005845.2_ASM584v2_feature_count.txt.gz\r\n-r--r--r--', '1', 'ftp', 'anonymous', '230172', 'Oct', '14', '2018', 'GCF_000005845.2_ASM584v2_feature_table.txt.gz\r\n-r--r--r--', '1', 'ftp', 'anonymous', '1379902', 'Oct', '31', '2014', 'GCF_000005845.2_ASM584v2_genomic.fna.gz\r\n-r--r--r--', '1', 'ftp', 'anonymous', '3462090', 'Mar', '2', '2019', 'GCF_000005845.2_ASM584v2_genomic.gbff.gz\r\n-r--r--r--', '1', 'ftp', 'anonymous', '465467', 'Feb', '11', '20:00', 'GCF_000005845.2_ASM584v2_genomic.gff.gz\r\n-r--r--r--', '1', 'ftp', 'anonymous', '483693', 'Feb', '11', '20:

And now that we have this as a list, we can simply parse for the filename that we want. The unique substring in this case is going to be 'cds_from_genomic'.

In [ ]:
for item in items:
    if 'cds_from_genomic' in item:
        file = item
        break
print(file)

GCF_000005845.2_ASM584v2_cds_from_genomic.fna.gz
-r--r--r--


Whoops, apparently there are newlines too, which wouldn't have separated the last element of a line from the first of the next. We can easily get rid of that though:

In [ ]:
# split string by newline and take the first part
file = file.split('\n')[0]
print(file)

GCF_000005845.2_ASM584v2_cds_from_genomic.fna.gz


And finally, we'll concatenate this filename with the url of the FTP directory, since we need the whole path to download it.

In [ ]:
file = ftp_url + '/' + file
print(file)

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_cds_from_genomic.fna.gz


#### Downloading files with wget

So we have our sequence length and we have the path to the file we'd like to download. We can do this using wget, a tool that comes with most terminals. There's also a Python package that you can install as follows.

In [ ]:
# installation command for miniconda/anaconda environment
conda install -c anaconda wget

Now literally all we have to do is call wget(url) to download the file into our current directory. I'm going to use the normal wget with os.system, since this is the more common practice. os is a default Python package and os.system allows you to run command line commands through Python.

In [ ]:
import os

os.system('wget ' + file)

2048

There seems to be an issue when I try to wget, and the out of 2048 tells me this (in general, anything nonzero is probably not right). The log on my command line looks like this:
![](https://drive.google.com/uc?id=1e38dOeM6vn1lN6Y1d446ZA8BI9wZr5Ph)

It seems that there was an extra '\r' on the end of our string, which is the return character. This was probably just included in the data format that urllib returned (which we can confirm by looking back), and we can easily get rid of this by slicing off the last character.

In [ ]:
file = file[:-1]
print(file)
os.system('wget ' + file)

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_cds_from_genomic.fna.gz


0

And we get an output of 0, so we're good! If you check in your current directory you should also see the file there. Note that the file we've downloaded is a gzip file (compressed format for sharing), so we'll have to unzip it. We can do this with the 'gunzip' command (terminal command) and os.system.

In [ ]:
# get filename back from whole path, and take last part of the path (actual filename)
print(file)
zip_name = file.split('/')[-1]
print(zip_name)
# unzip
os.system('gunzip ' + zip_name)

ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_cds_from_genomic.fna.gz
GCF_000005845.2_ASM584v2_cds_from_genomic.fna.gz


0

Another 0 output, looks good. We'll have to slice zip_name to get rid of the .gz (gunzip automatically names the new file without .gz), and then we can read the file in and show some of it to get an idea of what the data looks like. Once we've seen the data, we can identify some unique patterns and use that to filter out what we want.

In [ ]:
# chop .gz off of our filename
data_name = zip_name[:-3]
print(data_name)

GCF_000005845.2_ASM584v2_cds_from_genomic.fna


In [ ]:
lines = []
with open(data_name, 'r') as f:
    lines = f.readlines()
# preview
for line in lines[:10]:
    print(line)

>lcl|NC_000913.3_cds_NP_414542.1_1 [gene=thrL] [locus_tag=b0001] [db_xref=UniProtKB/Swiss-Prot:P0AD86] [protein=thr operon leader peptide] [protein_id=NP_414542.1] [location=190..255] [gbkey=CDS]

ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGA

>lcl|NC_000913.3_cds_NP_414543.1_2 [gene=thrA] [locus_tag=b0002] [db_xref=UniProtKB/Swiss-Prot:P00561] [protein=fused aspartate kinase/homoserine dehydrogenase 1] [protein_id=NP_414543.1] [location=337..2799] [gbkey=CDS]

ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAA

TGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTGAAAAAA

CCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGACGGGACTCGCCGCC

GCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATGTCCTGCATGG

CATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGCTGATTTGCCGTGGCGAGAAAATGTCGATCGCCATTA

TGGCCGGCGTATTAGAAGCGCGCGGTCACAACGTTACTGTTATCGATCCGGTCGAAAAACTGCTGGCAGTGGGGCATTAC

CTCGAATCTACCGTCGATATT

This data is in what is called fasta format (.fna). It has a pretty simple structure: for each entry, there is a header (denoted by a '>' at the start), and then lines of sequence follow starting on the next line. We can separate out headers and their respective sequences by parsing. If we were interested in the actual sequences we could log them, but since we only want lengths in this case, we'll make a dictionary that goes from header : sequence length. The unique identifier in this case is the '>', which tells us if a line is a header or a sequence.

In [ ]:
genes = {}

curr_header = None
curr_length = 0
# this file is really long! Let's only do the first 100 lines.
for line in lines[:100]:
    # check if a header. if yes, add last header to the dictionary and reset with new header and 0 length
    if line[0] == '>':
        # add the last entry if this is not the first iteration
        if curr_header != None and curr_length != 0:
            genes[curr_header] = curr_length
        # don't want to keep the '>' or the newline 
        curr_header = line[1:-1]
        curr_length = 0
    # we have a sequence, so get length (chopping off newline at end) and add it
    else:
        curr_length += len(line[:-1])
        
# show
for header in genes.keys():
    print(header)
    print(genes[header])

lcl|NC_000913.3_cds_NP_414542.1_1 [gene=thrL] [locus_tag=b0001] [db_xref=UniProtKB/Swiss-Prot:P0AD86] [protein=thr operon leader peptide] [protein_id=NP_414542.1] [location=190..255] [gbkey=CDS]
66
lcl|NC_000913.3_cds_NP_414543.1_2 [gene=thrA] [locus_tag=b0002] [db_xref=UniProtKB/Swiss-Prot:P00561] [protein=fused aspartate kinase/homoserine dehydrogenase 1] [protein_id=NP_414543.1] [location=337..2799] [gbkey=CDS]
2463
lcl|NC_000913.3_cds_NP_414544.1_3 [gene=thrB] [locus_tag=b0003] [db_xref=UniProtKB/Swiss-Prot:P00547] [protein=homoserine kinase] [protein_id=NP_414544.1] [location=2801..3733] [gbkey=CDS]
933
lcl|NC_000913.3_cds_NP_414545.1_4 [gene=thrC] [locus_tag=b0004] [db_xref=UniProtKB/Swiss-Prot:P00934] [protein=threonine synthase] [protein_id=NP_414545.1] [location=3734..5020] [gbkey=CDS]
1287
lcl|NC_000913.3_cds_NP_414546.1_5 [gene=yaaX] [locus_tag=b0005] [db_xref=UniProtKB/Swiss-Prot:P75616] [protein=DUF2502 domain-containing protein YaaX] [protein_id=NP_414546.1] [location=523

Perfect! We now have a dictionary containing the headers mapped to the length of their respective sequences. 

Looking at the headers, we see that different pieces of information are separated by square brackets and have some sort of attribute to describe them, such as 'gene' or 'location'. You know what that means...we can parse it! When we want to find the length for a particular gene, we can just parse the headers for the gene name and then get the corresponding sequence length. I encourage you to try using what you've learned and parsing out the gene names. Now that we've extracted all of the data we needed, we can write a simple Python script to calculate relative standard deviations and compare them. We won't cover this since it's relatively straightforward with some coding background, but it would be a great exercise if you'd like to get some practice writing a script to scrape and compare data *en masse*.

#### Wrapping up

In this tutorial and the last, we learned what web scraping was, how to locate data of interest on a webpage, and how to parse both HTML and other data formats to extract the important parts! I hope you've learned a lot and encourage you to try it yourself. If you don't have any webpage/data in mind, feel free to use the examples I suggested at the beginning of the first tutorial. I find that contriving some motivating purpose (i.e. some sort of data analysis after you've extracted it) makes learning to scrape a lot more fun too!

You are now free of the confines of nice, organized data--you can find and organize it yourself--and the massive amounts of data on the web are now yours to extract and analyze. Good luck, and happy scraping!

In [ ]:
%load_ext watermark
%watermark -v -p bs4,requests,urllib3,re

CPython 3.7.7
IPython 7.15.0

bs4 4.9.1
requests 2.23.0
urllib3 1.25.9
re 2.2.1
